In [ ]:
class SomeState:
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.values_list = []
        self.last_value = 0
        self.value_sum = 0

    def inc(self, d, row_id):
        self.last_value += d
        self.value_sum += self.last_value
        self.values_list.append(self.last_value)

    def set(self, v, row_id):
        self.last_value = v
        self.value_sum += self.last_value
        self.values_list.append(self.last_value)

s = StateLogger(SomeState())
s.set(v=10, row_id=10)
for i in range(9):
    s.inc(d=1, row_id=(i + 2) * 10)

print(s.state.values_list)

r = StateReplayer(s.log, SomeState)

r.step_forward()
r.step_forward()
r.step_forward()
r.step_forward()
r.step_forward()
r.step_forward()
r.step_forward()
r.step_forward()
r.step_forward()
assert not r.reached_end()

r.step_forward()
assert len(r.state.values_list) == 10
assert r.reached_end()

try:
    r.step_forward()
    assert False, 'step_forward should raise above'
except ReachedLogReplayEndException:
    pass


assert len(r.state.values_list) == 10

r.replay_to_end()
assert len(r.state.values_list) == 10

r.replay_to_before_row_id(0)
assert len(r.state.values_list) == 0
r.replay_to_before_row_id(10)
assert len(r.state.values_list) == 0
r.replay_to_before_row_id(11)
assert len(r.state.values_list) == 1
r.replay_to_before_row_id(30)
assert len(r.state.values_list) == 2
r.replay_to_before_row_id(100)
assert len(r.state.values_list) == 9

try:
    r.replay_to_before_row_id(101)
    assert False, 'step_forward should raise above'
except ReachedLogReplayEndException:
    pass

r.replay_to_before_row_id(0)
assert len(r.state.values_list) == 0
r.replay_to_end()
assert len(r.state.values_list) == 10
r.reset()
assert len(r.state.values_list) == 0
r.replay_to_before_row_id(1)
assert len(r.state.values_list) == 0
r.replay_to_after_row_id(1)
assert len(r.state.values_list) == 0
r.replay_to_after_row_id(95)
assert len(r.state.values_list) == 9
assert not r.reached_end()
r.step_forward()
assert r.reached_end()

r.replay_to_after_row_id(0)
assert len(r.state.values_list) == 0
r.replay_to_after_row_id(50)
assert len(r.state.values_list) == 5
r.replay_to_after_row_id(10)
assert len(r.state.values_list) == 1
r.replay_to_before_row_id(20)
assert len(r.state.values_list) == 1

try:
    r.replay_to_after_row_id(100)
    assert False, 'replay_to_after_row_id should raise above'
except ReachedLogReplayEndException:
    pass


r.state.values_list